In [23]:
# Useful for debugging
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
from gpt.gpt import GPT 

In [25]:
G = GPT(input_file='/Users/colwyngulliford/Documents/GitHub/lume-gpt/templates/dcgun/gpt.in')